In [ ]:
# BLAST - RNF130
from Bio import SeqIO
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio import SearchIO


def get_prot(id):
    """
    id: Código da proteína a pesquisar na base de dados Swiss-Prot ("swiss")
    
    Retorna uma lista de Accession numbers filtrados em gb e sem repetições
    """
    from Bio import ExPASy
    with ExPASy.get_sprot_raw(id) as handle:
        seq_record = SeqIO.read(handle, "swiss")
    print(seq_record.seq)
    print(len(seq_record.seq), 'aa')
    for k, v in seq_record.annotations.items():
        print(k, v)
    tam= len(seq_record.seq)
    seq= seq_record.seq
    tax= seq_record.annotations["taxonomy"]
    org= seq_record.annotations["organism"]
    y = ('ID:' + id + '|' + 'SEQUENCE:' + seq + '|' + 'SEQUENCE LENGTH:' + str(tam) + 'bp' + '|' + 'TAXONOMY:' + str(tax) + '|' + 'ORGANISM:' + org )
    return y

def filtro(seq):
    """ 
    seq: string resultante do get_prot(id)
    
    Retorna uma lista da string separada pelo caractere "|"
    """
    seq = seq.split('|') # Separador 
    return seq

def blast_prot(file, seq):
    """
    file: Nome do ficheiro a dar com o resultado do BLAST
    seq: sequência query para executar o BLAST
    
    Retorna uma lista dos resultados do BLAST com o nome do alinhamento, comprimento do alinhamento e e-value 
    (filtrado pelo E-Value)
    """
    result_handle = NCBIWWW.qblast('blastp', 'nr', seq)
    with open(file, "w") as out_handle:
        out_handle.write(result_handle.read())
    result_handle.close()

def obter_homologos(file, E_VALUE_THRESH):
    """
    file: Nome do ficheiro a dar com o resultado do BLAST
    E_VALUE_THRESH: Recebe um valor "None" ou qualquer número int, corresponde ao e-value máximo aceitável para 
    tratamento do output do BLAST
    
    Retorna uma lista dos resultados do BLAST com o nome do alinhamento, comprimento do alinhamento e e-value 
    (filtrado pelo E-Value)
    """
    result_handle = open(file)
    blast_record = NCBIXML.read(result_handle)
    FILE = str('SeqsHomologas_' + str(file) + '.fasta')
    ficheiro_output = open(FILE, 'w+')
    if E_VALUE_THRESH == None:
        E_VALUE_THRESH = 0.05
    for alignment in blast_record.alignments:
        for hsp in range(len(alignment.hsps)):
            if alignment.hsps[hsp].expect < E_VALUE_THRESH:
                if hsp != 0:
                    pass
                else:
                    ficheiro_output.write('>' + alignment.title + '\n' + alignment.hsps[hsp].sbjct + '\n')
    result_handle.close()

def parse_prot(file):
    """
    file: Nome do ficheiro a dar com o resultado do BLAST
    
    Retorna informações contidas no ficheiro
    """
    result_handle = open(file)
    from Bio.Blast import NCBIXML
    blast_record = NCBIXML.read(result_handle)
    E_VALUE_THRESH = 0.04
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            if hsp.expect < E_VALUE_THRESH:
                print("****Alignment****")
                print("sequence:", alignment.title)
                print("length:", alignment.length)
                print("e value:", hsp.expect)
                print(hsp.query[0:75] + "...")
                print(hsp.match[0:75] + "...")
                print(hsp.sbjct[0:75] + "...")
    from Bio import SearchIO
    blast_qresult = SearchIO.read(file, "blast-xml")
    print(blast_qresult)
    result_handle.close()


def proteico(id, file, blast = False, E_VALUE_THRESH = None):
    """
    genbank: ficheiro genbank (formato .gb)
    id: id Swiss-Prot da proteína
    file: Nome do ficheiro a dar com o resultado do BLAST
    blast: Em caso de omissão, blast = False; se blast = True, irá recorrer ao ficheiro .gb para realizar o BLAST
    E_VALUE_THRESH: Recebe um valor "None" ou qualquer número int, corresponde ao e-value máximo aceitável para 
    tratamento do output do BLAST

    Gera um ficheiro .txt contendo os Accession Numbers dos resultados do BLAST com filtração segundo o E_VALUE_THRESH e gb
    """
    if blast == True:
        from Bio import ExPASy
        with ExPASy.get_sprot_raw(id) as handle:
            seq_record = SeqIO.read(handle, "swiss")
        blast_prot(file, seq_record.seq)
    x = get_prot(id)
    print(x)
    obter_homologos(file, E_VALUE_THRESH)
    parse_prot(file)

proteico('Q86XS8', 'RNF130_blast.xml', True, 0.05)